<a href="https://colab.research.google.com/github/MDiakhate12/data_engineer_assignment/blob/main/Data_Engineer_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark==3.1.3

  Using cached pyspark-3.1.3.tar.gz (214.0 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached py4j-0.10.9-py2.py3-none-any.whl (198 kB)
Using legacy 'setup.py install' for pyspark, since package 'wheel' is not installed.
  Running setup.py install for pyspark: started
  Running setup.py install for pyspark: finished with status 'done'


You should consider upgrading via the 'C:\Users\mdiakhat\OneDrive - Capgemini\Documents\data_engineering_assignment\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
!pip install python-dotenv

You should consider upgrading via the 'C:\Users\mdiakhat\OneDrive - Capgemini\Documents\data_engineering_assignment\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
! pip install requests

     ---------------------------------------- 62.8/62.8 KB 1.7 MB/s eta 0:00:00
  Using cached idna-3.3-py3-none-any.whl (61 kB)
     -------------------------------------- 160.2/160.2 KB 1.9 MB/s eta 0:00:00
  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)


You should consider upgrading via the 'C:\Users\mdiakhat\OneDrive - Capgemini\Documents\data_engineering_assignment\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 

import os
from os.path import splitext
from os.path import join, dirname

from dotenv import load_dotenv

In [ ]:
INPUT_FORMAT = "parquet"

DATE = "2020-01"

BASE_DATASET_NAME = "yellow_tripdata"

DATA_ROOT_PATH = join(dirname('__file__'), "data")

FILENAME = f"{BASE_DATASET_NAME}_{DATE}.{INPUT_FORMAT}"

BASE_URL = "https://s3.amazonaws.com/nyc-tlc/trip+data"

DATASET_URL = f"{BASE_URL}/{FILENAME}"

PARTITIONNED_FILENAME = f"{splitext(FILENAME)[0]}_partitionned"


# List of maven coordinate jar packages 
JAR_PACKAGES = [
                "com.microsoft.azure:spark-mssql-connector_2.12:1.2.0", 
                "com.microsoft.sqlserver:mssql-jdbc:10.2.0.jre8",
                ]

In [ ]:
print("INPUT_FORMAT:", INPUT_FORMAT)
print("DATE:", DATE)
print("BASE_DATASET_NAME:", BASE_DATASET_NAME)
print("FILENAME:", FILENAME)
print("BASE_URL:", BASE_URL)
print("DATASET_URL:", DATASET_URL)
print("PARTITIONNED_FILENAME:", PARTITIONNED_FILENAME)
print("JAR_PACKAGES:", JAR_PACKAGES)
print("DATA_ROOT_PATH:", DATA_ROOT_PATH)

INPUT_FORMAT: parquet
DATE: 2020-01
BASE_DATASET_NAME: yellow_tripdata
FILENAME: yellow_tripdata_2020-01.parquet
BASE_URL: https://s3.amazonaws.com/nyc-tlc/trip+data
DATASET_URL: https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-01.parquet
PARTITIONNED_FILENAME: yellow_tripdata_2020-01_partitionned
JAR_PACKAGES: ['com.microsoft.azure:spark-mssql-connector_2.12:1.2.0', 'com.microsoft.sqlserver:mssql-jdbc:10.2.0.jre8']
DATA_ROOT_PATH: data


In [ ]:
spark = SparkSession \
  .builder \
  .master("local[*]") \
  .appName("Data Engineer Assignment") \
  .config("spark.jars.packages", ",".join(JAR_PACKAGES)) \
  .getOrCreate()

In [ ]:
# Showing the spark version

print(spark.sparkContext.version)

3.1.3


In [ ]:
# Check if libraries has been correctly installed

print(spark.sparkContext._jsc.sc().listJars())

Vector(spark://LFR028606.corp.capgemini.com:50061/jars/com.microsoft.azure_spark-mssql-connector_2.12-1.2.0.jar, spark://LFR028606.corp.capgemini.com:50061/jars/com.microsoft.sqlserver_mssql-jdbc-10.2.0.jre8.jar)


In [ ]:
import requests

In [ ]:
# ! rm -rf ${BASE_DATASET_NAME}*
# ! ls ${BASE_DATASET_NAME}*
# ! wget $DATASET_URL

response = requests.get(DATASET_URL)

with open(f"{DATA_ROOT_PATH}/{FILENAME}", "wb") as file:
    file.write(response.content)

In [ ]:
df = spark.read.parquet(f"{DATA_ROOT_PATH}/{FILENAME}")

In [ ]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)



# 1. Imagine that you receive data in single day batches, based on the field “tpep_dropoff_datetime”. Split the .csv file into multiple files based on this field. You should end up in roughly 30 files. Consider using a more efficient file format than csv.

In [ ]:
# Overview of data 

df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2020-01-01 01:28:15|  2020-01-01 01:33:03|            1.0|          1.2|       1.0|                 N|         238|         239|           1|        6.0|  3.0|    0.5|      1.4

In [ ]:
# Count the number of datetime values

df.select("tpep_dropoff_datetime").distinct().count()

2137286

In [ ]:
# Quick check of some data values

df.select("tpep_dropoff_datetime", "VendorID", "trip_distance", "fare_amount").show(5)

+---------------------+--------+-------------+-----------+
|tpep_dropoff_datetime|VendorID|trip_distance|fare_amount|
+---------------------+--------+-------------+-----------+
|  2020-01-01 01:33:03|       1|          1.2|        6.0|
|  2020-01-01 01:43:04|       1|          1.2|        7.0|
|  2020-01-01 01:53:52|       1|          0.6|        6.0|
|  2020-01-01 02:00:14|       1|          0.8|        5.5|
|  2020-01-01 01:04:16|       2|          0.0|        3.5|
+---------------------+--------+-------------+-----------+
only showing top 5 rows



In [ ]:
# Checking ordered distinct values of the "tpep_dropoff_datetime" field

df \
  .select(dayofmonth("tpep_dropoff_datetime").alias("dayofmonth")) \
  .distinct() \
  .orderBy("dayofmonth") \
  .show()

+----------+
|dayofmonth|
+----------+
|         1|
|         2|
|         3|
|         4|
|         5|
|         6|
|         7|
|         8|
|         9|
|        10|
|        11|
|        12|
|        13|
|        14|
|        15|
|        16|
|        17|
|        18|
|        19|
|        20|
+----------+
only showing top 20 rows



In [ ]:
# Counting the number of distinct values into the "tpep_dropoff_datetime" field

df.select(dayofmonth("tpep_dropoff_datetime")).distinct().count()

31

In [ ]:
DATA_ROOT_PATH

'data'

In [ ]:
# Partition and save the date data by day for one month 

# We choose to save file into parquet format which is optimized for read 

df \
  .withColumn("dayofmonth", dayofmonth("tpep_dropoff_datetime")) \
  .write \
  .mode("overwrite") \
  .partitionBy("dayofmonth") \
  .parquet(f"./data/yellow_tripdata_2020-01_partitionned")

In [ ]:
# Check if the data is correctly saved  

# !ls -al $PARTITIONNED_FILENAME

total 140
drwxr-xr-x 33 root root 4096 Jun 21 11:34  .
drwxr-xr-x  1 root root 4096 Jun 21 11:33  ..
drwxr-xr-x  2 root root 4096 Jun 21 11:33 'dayofmonth=1'
drwxr-xr-x  2 root root 4096 Jun 21 11:34 'dayofmonth=10'
drwxr-xr-x  2 root root 4096 Jun 21 11:34 'dayofmonth=11'
drwxr-xr-x  2 root root 4096 Jun 21 11:34 'dayofmonth=12'
drwxr-xr-x  2 root root 4096 Jun 21 11:34 'dayofmonth=13'
drwxr-xr-x  2 root root 4096 Jun 21 11:34 'dayofmonth=14'
drwxr-xr-x  2 root root 4096 Jun 21 11:34 'dayofmonth=15'
drwxr-xr-x  2 root root 4096 Jun 21 11:34 'dayofmonth=16'
drwxr-xr-x  2 root root 4096 Jun 21 11:34 'dayofmonth=17'
drwxr-xr-x  2 root root 4096 Jun 21 11:34 'dayofmonth=18'
drwxr-xr-x  2 root root 4096 Jun 21 11:34 'dayofmonth=19'
drwxr-xr-x  2 root root 4096 Jun 21 11:34 'dayofmonth=2'
drwxr-xr-x  2 root root 4096 Jun 21 11:34 'dayofmonth=20'
drwxr-xr-x  2 root root 4096 Jun 21 11:34 'dayofmonth=21'
drwxr-xr-x  2 root root 4096 Jun 21 11:34 'dayofmonth=22'
drwxr-xr-x  2 root root 4096 Ju

# 2. Your next task is to implement a process that loads all of the files into the database. It would be beneficial that the implementation is reusable, i.e.can be used for the future data files as well.

In [ ]:
# Showing the dotenv file created with Azure SQL DB credentials

!ls -al .env*

-rw-r--r-- 1 root root 159 Jun 21 09:53 .env


In [ ]:
dotenv_path = join(dirname('__file__'), '.env')
print(dotenv_path)

.env


In [ ]:
load_dotenv(dotenv_path)

True

In [ ]:
# Azure SQL DB Database Credential
HOSTNAME = os.environ.get("HOSTNAME")
USER = os.environ.get("USER")
PASSWORD = os.environ.get("PASSWORD")
DATABASE = os.environ.get("DATABASE")
PORT = os.environ.get("PORT")
SERVER = os.environ.get("SERVER")

# SQLServer Driver
DRIVER = "com.microsoft.sqlserver.jdbc.SQLServerDriver" 

# Azure SQL Connector JDBC Format
SQL_SERVER_SPARK_JDBC_FORMART = "com.microsoft.sqlserver.jdbc.spark"

# Database Connection String
URL =  f"jdbc:sqlserver://{HOSTNAME}:{PORT};databaseName={DATABASE};"

In [ ]:
reader = spark.read.format(SQL_SERVER_SPARK_JDBC_FORMART) \
    .option("url", URL) \
    .option("user", USER) \
    .option("driver", DRIVER) \
    .option("password", PASSWORD) 

In [ ]:
df_partitionned = spark.read.parquet(PARTITIONNED_FILENAME)

In [ ]:
df_partitionned.show(3)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|dayofmonth|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+----------+
|       2| 2020-01-16 23:34:00|  2020-01-17 00:08:00|           null|         14.1|      null|              null|         136|          90|           0|  

In [ ]:
df_partitionned.write.format(SQL_SERVER_SPARK_JDBC_FORMART) \
    .option("mode", "overwrite") \
    .option("url", URL) \
    .option("user", USER) \
    .option("driver", DRIVER) \
    .option("password", PASSWORD) \
    .option("dbtable", f"{BASE_DATASET_NAME}_{DATE}") \
    .save()

# 3. Test the quality of the data. What metrics would you use to monitor the quality of the data? Are there any anomalous or suspicious data points in the data? We would like to collect data quality metrics of processed data each time it is loaded into the database. Could you implement this functionality and store such metrics in a separate database table or any other place that you seem fit?

# 4. Model the dataset to represent a fact-dimension type of schema. Draw the entity relationship diagram of your model and transform the data to fit it.

# 5. Can you identify the weak points of the proposed solution? If you had more time, what parts of your solution would you improve?

# Weak points and Imporvements

1.  The code should be written with functions and parameterized to be more reusable and easier to test (unit tests and integration tests)
2.  We should use a defined schema for the data instead of inferring it: this enables the data loading to be faster and more robust and also prevent any type issue
3.  We should use try-catch blocks to prevent our data pipeline from crashing by catching the corresponding exceptions 
4.  The data must be ingested in a storage in the cloud or locally in a more secure data storage and decoupled from the code in order to prevent any data loss
5.  We can decouple data ingestion and data transformation to have better modularity
6.  By using the decoupling mehod we can use Kafka to ingest data in an event based way
7.  We can use KNative to deploy our pipeline as a serverless container. In this way, the container can be configured to be triggered only when new data arrives and then free ressources after that. The requirement for that is to have a Kubernetes cluster running either in a cloud provider or locally
8.  After my experience with fact-dimension model, I should instead go for a single large table such as Big Query. This makes requests easier without the need to manage many foreign keys which require join requests.
9.  This code is just a notebook for demonstration purposes. But the real code would be packaged in a python file and submitted via spark-submit and/or dockerized
10. To test the quality of the data, we can refer to thoses five metrics: acccuracy (correct number scale and precisions), validity (correct data types and format), completeness (null values, zero values), uniqueness (correct integrity constrains) and conformity with the data type and business requirements
11. We can use the pandas-profiling library to profile the data and get some useful metrics and prometheus with grafana to query and show more complex metrics
12. With simple SQL requests we can create tables and ingest data and so some simple checks 
13. We can automate all the pipeline either by using simple cron jobs or by going more robust by using apache airflow
